Ref: 
(https://www.kaggle.com/code/ambrosm/tpsaug22-eda-which-makes-sense)

In [ ]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
import seaborn as sns
import warnings
from colorama import Fore, Back, Style
import scipy.stats

from sklearn.model_selection import GroupKFold
from sklearn.preprocessing import OneHotEncoder, OrdinalEncoder, StandardScaler
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import SimpleImputer, IterativeImputer, KNNImputer
from sklearn.ensemble import RandomForestClassifier, ExtraTreesClassifier
from sklearn.pipeline import make_pipeline
from sklearn.linear_model import LogisticRegression, LinearRegression
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis, LinearDiscriminantAnalysis
from sklearn.metrics import roc_auc_score, roc_curve

from lightgbm import LGBMClassifier, early_stopping, log_evaluation

np.set_printoptions(linewidth=150)

## Data Loading

In [ ]:
train = pd.read_csv('../data/01_raw/tabular-playground-series-aug-2022/train.csv',
                    index_col='id')
test = pd.read_csv('../data/01_raw/tabular-playground-series-aug-2022/test.csv',
                    index_col='id')
display(train)
display(test)
both = pd.concat([train[test.columns], test])

Of the 26,570 from Train set, only 21 percent failed. 

<b>Insights</b>: This looks slightly imbalanced. Might be good to stratify train-test split.

In [ ]:
train.failure.value_counts() / len(train)

In [ ]:
# Check missing value 
mis_train = train.isna().sum().to_frame()#.reset_index()
mis_test = test.isna().sum().to_frame()
mis_train.columns, mis_test.columns = ['train_cols'], ['test_cols']
missing_cols = pd.concat([mis_train, mis_test], axis=1)
missing_cols

In [ ]:
train.info()

### Float columns

In [ ]:
# float columns.
float_cols = [f for f in train.columns if train[f].dtype == float]

In [ ]:
_, axs = plt.subplots(4, 4, figsize=(12,12))
for f, ax in zip(float_cols, axs.ravel()):
    mi = min(train[f].min(), test[f].min())
    ma = max(train[f].max(), test[f].max())
    bins = np.linspace(mi, ma, 50)
    ax.hist(train[f], bins=bins, alpha=0.5, density=True, label='train')
    ax.hist(test[f], bins=bins, alpha=0.5, density=True, label='test')
    ax.set_xlabel(f)
    if ax == axs[0, 0]: ax.legend(loc='lower left')
        
    ax2 = ax.twinx()
    total, _ = np.histogram(train[f], bins=bins)
    failures, _ = np.histogram(train[f][train.failure == 1], bins=bins)
    with warnings.catch_warnings(): # ignore divide by zero for empty bins
        warnings.filterwarnings('ignore', category=RuntimeWarning)
        ax2.scatter((bins[1:] + bins[:-1]) / 2, failures / total,
                    color='m', s=10, label='failure probability')
    ax2.set_ylim(0, 0.5)
    ax2.tick_params(axis='y', colors='m')
    if ax == axs[0, 0]: ax2.legend(loc='upper right')
plt.tight_layout(w_pad=1)
plt.suptitle('Train and test distributions of the continuous features', fontsize=20, y=1.02)
plt.show()

Insight: With that many missing values, good value imputation will be the key for winning the competition.

Let's look at the distribution of the float features. We plot the train and test histograms in the same diagram - train is blue, test is orange.

We see that the first feature, loading, has a skewed distribution, perhaps log-normal; the other features are normally distributed. The first eight features have the same distribution in train and test; from measurement_10 onwards the distributions differ slightly.

In the same diagrams, we show the failure probabilities with magenta dots. The top left diagram shows that higher loadings imply a higher failure probability. The bottom right diagram shows that measurement_17 is positively correlated to the target as well. All other float features seem to be uncorrelated to the failure probability (the failure probability is 21 % independent of the feature value).

#### Binomial distribution -> test for failure count.

Maybe the cause of a product failure triggers the failure of a measurement device. How can we test this idea? We calculate the conditional product failure rate given the measurement is missing E[product fails | measurement is missing] and compare it to the unconditional product failure rate, which is 0.212608.

In [ ]:
# Start by plotting the bell curve
plt.figure(figsize=(12, 4))
z_ticks = np.linspace(-3.5, 3.5, 61)
pdf = scipy.stats.norm.pdf(z_ticks)
plt.plot(z_ticks, pdf)
# Calculate the conditional failure rate for every missing feature
# Print the values and plot them
print('feature           fail   miss   failure rate       z    p-value')
for f in train.columns:
    if train[f].isna().sum() > 0:
        total = train[f].isna().sum()
        fail = train[train[f].isna()].failure.sum()
        z = (fail / total - 0.212608) / (np.sqrt(0.212608 * (1-0.212608)) / np.sqrt(total))
        plt.scatter([z], [scipy.stats.norm.pdf(z)], c='r' if abs(z) > 2 else 'g', s=100)
        print(f"{f:15} : {fail:4} / {total:4} = {fail/total:.3f}          {z:5.2f}      {2*scipy.stats.norm.cdf(-abs(z)):.3f}")
        if abs(z) > 1: plt.annotate(f"{f}: {fail / total:.3f}",
                                    (z, scipy.stats.norm.pdf(z)),
                                    xytext=(0,10), 
                                    textcoords='offset points', ha='left' if z > 0 else 'right',
                                    color='r' if abs(z) > 2 else 'g')
            
# Annotage the center (z=0)
plt.vlines([0], 0, 0.05, color='g')
plt.annotate(f"z_score = 0\naverage failure rate: {0.212608:.3f}",
                                    (0, 0.05),
                                    xytext=(0,10), 
                                    textcoords='offset points', ha='center',
                                    color='g')
plt.title('Failure rate when feature is missing')
plt.yticks([])
plt.xlabel('z_score')
plt.show()

When measurement_3 is missing, the failure rate is 0.160 (much lower than average).
When measurement_5 is missing, the failure rate is 0.254 (much higher than average).

### Integer columns

In [ ]:
int_cols = [f for f in train.columns if train[f].dtype == 'int64' and f != 'failure']
pd.concat([train[int_cols].isna().sum().rename('missing_values in train'),
           test[int_cols].isna().sum().rename('missing values in test')], 
          axis=1)

In [ ]:
# Let's plot the distributions of the five integer features for train and test:
_, axs = plt.subplots(2, 3, figsize=(12, 8))
for f, ax in zip(int_cols, axs.ravel()):
    temp1 = train.failure.groupby(train[f]).agg(['mean', 'size'])
    ax.bar(temp1.index, temp1['size'] / len(train), alpha=0.5, label='train')
    temp2 = test[f].value_counts()
    ax.bar(temp2.index, temp2 / len(test), alpha=0.5, label='test')
    ax.set_xlabel(f)
    ax.set_ylabel('frequency')
    
    ax2 = ax.twinx()
    ax2.scatter(temp1.index, temp1['mean'],
                color='m', label='failure probability')
    ax2.set_ylim(0, 0.5)
    ax2.tick_params(axis='y', colors='m')
    if ax == axs[0, 0]: ax2.legend(loc='upper right')

axs[0, 0].legend()
axs[1, 2].axis('off')
plt.tight_layout(w_pad=1)
plt.suptitle('Train and test distributions of the integer features', fontsize=20, y=1.02)
plt.show()
del temp1, temp2

We can see for attribute 2 values occur only in train set, 5 and 8, while attribute 7 only exists in test set. 
attribute 3 is also similar.
Measurement_2 has positive correlation to failure (target) for values >10. 

### The string columns

In [ ]:
string_cols = [f for f in train.columns if train[f].dtype == object]
pd.concat([train[string_cols].isna().sum().rename('missing values in train'),
           test[string_cols].isna().sum().rename('missing values in test')],
          axis=1) # no missing values.

In [ ]:
_, axs = plt.subplots(1, 3, figsize=(12, 4))
for f, ax in zip(string_cols, axs.ravel()):
    temp1 = train[f].value_counts(dropna=False, normalize=True)
    temp2 = test[f].value_counts(dropna=False, normalize=True)
    values = sorted(set(temp1.index).union(temp2.index))
    temp1 = temp1.reindex(values)
    temp2 = temp2.reindex(values)
    ax.bar(range(len(values)), temp1, alpha=0.5, label='train')
    ax.bar(range(len(values)), temp2, alpha=0.5, label='test')
    ax.set_xlabel(f)
    ax.set_ylabel('frequency')
    ax.set_xticks(range(len(values)), values)
    
    temp1 = train.failure.groupby(train[f]).agg(['mean', 'size'])
    temp1 = temp1.reindex(values)
    ax2 = ax.twinx()
    ax2.scatter(range(len(values)), temp1['mean'],
                color='m', label='failure probability')
    ax2.tick_params(axis='y', colors='m')
    ax2.set_ylim(0, 0.5)
    if ax == axs[0]: ax2.legend(loc='lower right')

axs[0].legend()
plt.suptitle('Train and test distributions of the string features', fontsize=20, y=0.96)
plt.tight_layout(w_pad=1)
plt.show()
del temp1, temp2  

Product codes do not overlap between train and test set. To simulate failure, we have to simulate this situation by splitting the data so that the validation set contains other products than the training set. 

Attribute_0 and attribute_1 are categorical features which should be one-hot encoded.

### Missing data analysis

In [ ]:
import missingno as msno
Nrows,Ncols = train.shape

msno.matrix(train.iloc[np.random.choice(range(Nrows), 250)])

The missingness seems similar between both the training and the test set. The product_code and the `attribute_i` features are all completely non-missing. `measurement_i`, for  i≤2  are all non-missing. `measurement_i` for  `i≥3`  have some missing values and have progressively increasing proportion of missingness. This suggests that there are a series of measurements, the results of which determine whether or not later tests are carried out. The probabilities of later measurements being missing doesn't appear to be too strongly related to the missingness of earlier measurements, as inferred from msno.heatmap (not shown here), however the dendrogram analysis from msno shows a definite pattern:

In [ ]:
msno.dendrogram(train)

In [ ]:
# Product codes and attributes
both[string_cols + ['attribute_2', 'attribute_3']].drop_duplicates().set_index('product_code')

<iframe src="https://www.kaggle.com/embed/ambrosm/tpsaug22-eda-which-makes-sense?cellIds=28&kernelSessionId=102683452" height="300" style="margin: 0 auto; width: 100%; max-width: 950px;" frameborder="0" scrolling="auto" title="TPSAUG22 EDA which makes sense ⭐️⭐️⭐️⭐️⭐️"></iframe>

In [ ]:
train.head()

In [ ]:
auc_list = []
test_pred_list = []
importance_list = []

kf = GroupKFold(n_splits=5)
for fold, (idx_tr,idx_va) in enumerate(kf.split(train, train.failure, train.product_code)):
    X_tr = train.iloc[idx_tr][test.columns]
    X_va = train.iloc[idx_va][test.columns]
    X_te = test.copy()
    y_tr = train.iloc[idx_tr].failure
    y_va = train.iloc[idx_va].failure
    
    # one-hot encode attribute_0 and attribute_1
    ohe_attributes = ['attribute_0', 'attribute_1']
    ohe_output = ['ohe0_7', 'ohe1_6', 'ohe1_8']
    ohe = OneHotEncoder(categories=[['material_5', 'material_7'],
                                    ['material_5', 'material_6', 'material_8']],
                        drop='first', sparse=False, handle_unknown='ignore')
    ohe.fit(X_tr[ohe_attributes])
    
    for df in [X_tr, X_va, X_te]:
        with warnings.catch_warnings(): # ignore "Found unknown categories"
            warnings.filterwarnings('ignore', category=UserWarning)
            df[ohe_output] = ohe.transform(df[ohe_attributes])
        df.drop(columns=ohe_attributes, inplace=True)
        
    # We add the indicators for missing values
    for df in [X_tr, X_va, X_te]:
        df['m_3_missing'] = df.measurement_3.isna()
        df['m_5_missing'] = df.measurement_5.isna()
    
    # We fill the missing values
    features = [f for f in X_tr.columns if f == 'loading' or f.startswith('measurement')]
    imputer = KNNImputer(n_neighbors=3)
    imputer.fit(X_tr[features])
    for df in [X_tr, X_va, X_te]:
        df[features] = imputer.transform(df[features])
    
    # The EDA diagram of measurement 2 shows that the feature is correlated
    # to the target only for values above 10. For this reason, we clip
    # all values below 11.
    for df in [X_tr, X_va, X_te]:
        df['measurement_2'] = df['measurement_2'].clip(11, None)
    
    # We fit a model
    features2 = [f for f in X_tr.columns if f != 'product_code']
    model = make_pipeline(StandardScaler(), 
                          LogisticRegression(penalty='l1', C=0.01,
                                             solver='liblinear', random_state=1))
    model.fit(X_tr[features2], y_tr)
    importance_list.append(model.named_steps['logisticregression'].coef_.ravel())
    
    # We validate the model
    y_va_pred = model.predict_proba(X_va[features2])[:,1]
    score = roc_auc_score(y_va, y_va_pred)
    print(f"Fold {fold}: auc = {score:.5f}")
    auc_list.append(score)
# Show overall score
print(f"{Fore.GREEN}{Style.BRIGHT}Average auc = {sum(auc_list) / len(auc_list):.5f}{Style.RESET_ALL}")

# Show feature importances
importance_df = pd.DataFrame(np.array(importance_list).T, index=features2)
importance_df['mean'] = importance_df.mean(axis=1).abs()
importance_df['feature'] = features2
importance_df = importance_df.sort_values('mean', ascending=False).reset_index().head(10)
plt.figure(figsize=(14, 4))
plt.barh(importance_df.index, importance_df['mean'], color='lightgreen')
plt.gca().invert_yaxis()
plt.yticks(ticks=importance_df.index, labels=importance_df['feature'])
plt.title('LogisticRegression feature importances')
plt.show()

In [ ]:
y_tr

## Classification with keras

https://www.kaggle.com/code/heyspaceturtle/feature-selection-is-all-u-need?scriptVersionId=102667189

In [ ]:
# Data Processing
import numpy as np
import pandas as pd
import random
from sklearn import preprocessing
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer
from sklearn.preprocessing import LabelEncoder

# Modeling 
from sklearn.ensemble import VotingClassifier
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import cluster, accuracy_score, roc_auc_score
from sklearn.model_selection import cross_validate, GridSearchCV, cross_val_score, StratifiedKFold

import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input
from tensorflow.keras.layers import Dense
from tensorflow.keras.utils import plot_model

# Other 
import warnings
warnings.filterwarnings('ignore')

In [ ]:
# Load Data
train = pd.read_csv('../data/01_raw//tabular-playground-series-aug-2022/train.csv')
test = pd.read_csv('../data/01_raw/tabular-playground-series-aug-2022/test.csv')
sample_submission = pd.read_csv('../data/01_raw//tabular-playground-series-aug-2022/sample_submission.csv')

train.head()

In [ ]:
# Variable lists for easy manipulation
id_var = ['id']
target= ['failure']
cat_vars = ['product_code','attribute_0','attribute_1']
num_vars = [v for v in test.columns if v not in id_var and v not in cat_vars]
predictors = cat_vars + num_vars

Data preprocessing

In [ ]:
# Track missing values 
#for v in num_vars: 
#    if train[v].isna().sum() > 0: 
#        train[f'na_{v}']= np.where(train[v].isna()==True, 1,0)
#        test[f'na_{v}']= np.where(test[v].isna()==True, 1,0)
        
#Imputing missing values 
multi_imp = IterativeImputer(max_iter = 9, random_state = 42, verbose = 0, skip_complete = True, n_nearest_features = 10, tol = 0.001)
multi_imp.fit(train[num_vars])
train[num_vars] = multi_imp.transform(train[num_vars])
test[num_vars] = multi_imp.transform(test[num_vars])

In [ ]:
# Drop product code
test = test.drop(['product_code'], axis = 1)
train = train.drop(['product_code'], axis = 1)
cat_vars.remove('product_code')

In [ ]:
# feature engineering
train['attribute_2*3'] = train['attribute_2'] * train['attribute_3']
test['attribute_2*3'] = test['attribute_2'] * test['attribute_3']

train['meas17_loading_ratio'] = train['measurement_17']/train['loading']
test['meas17_loading_ratio'] = test['measurement_17']/test['loading']

meas_cols = [f"measurement_{i:d}" for i in list(range(3, 17))]
train['meas_avg'] = np.mean(train[meas_cols], axis=1)
train['meas_std'] = np.std(train[meas_cols], axis=1)
train['meas_max'] = np.max(train[meas_cols], axis=1)
train['meas_min'] = np.min(train[meas_cols], axis=1)

test['meas_avg'] = np.mean(test[meas_cols], axis=1)
test['meas_std'] = np.std(test[meas_cols], axis=1) 
test['meas_max'] = np.max(test[meas_cols], axis=1)
test['meas_min'] = np.min(test[meas_cols], axis=1)

In [ ]:
# Label Encoding
label_encoder = LabelEncoder()
train_le = train.copy()
test_le = test.copy()

for col in ['attribute_0', 'attribute_1']:
    train_le[col] = label_encoder.fit_transform(train[col])
    test_le[col] = label_encoder.fit_transform(test[col]) 
        
train = train_le
test = test_le

In [ ]:
# update predictor list
predictors = [v for v in train.columns if v not in id_var and v not in target]
train.shape

y_class = LabelEncoder().fit_transform(train[target])

# Train test split
X_train, X_test, y_train, y_test, y_train_class, y_test_class = train_test_split(train[predictors], train[target], y_class, test_size=0.2, random_state=42)

### Feature selection with Fischer's score

In [ ]:
def FisherScore(bt, y_train, predictors):
    """
    Verbeke, W., Dejaeger, K., Martens, D., Hur, J., & Baesens, B. (2012). New insights
    into churn prediction in the telecommunication sector: A profit driven data mining
    approach. European Journal of Operational Research, 218(1), 211-229.
    """
    # Get the unique values of dependent variable
    target_var_val = y_train.unique()
    
    # Calculate FisherScore for each predictor
    predictor_FisherScore = []
    for v in predictors:
        fs = np.abs(np.mean(bt.loc[y_train == target_var_val[0], v]) - np.mean(bt.loc[y_train == target_var_val[1], v])) / \
             np.sqrt(np.var(bt.loc[y_train == target_var_val[0], v]) + np.var(bt.loc[y_train == target_var_val[1], v]))
        predictor_FisherScore.append(fs)
    return predictor_FisherScore

In [ ]:
# Calculate Fisher Score for all variables
fs = FisherScore(train, train['failure'], predictors)
fs_df = pd.DataFrame({"predictor":predictors, "fisherscore":fs})
fs_df = fs_df.sort_values('fisherscore', ascending=False).round(3)
fs_df.head()

In [ ]:
fs_cols = fs_df.head().predictor.tolist()
fs_cols

In [ ]:
# visualise FS head.
_, axs = plt.subplots(3, 2, figsize=(8,8))
for f, ax in zip(fs_cols, axs.ravel()):
    mi = min(train[f].min(), test[f].min())
    ma = max(train[f].max(), test[f].max())
    bins = np.linspace(mi, ma, 50)
    ax.hist(train[f], bins=bins, alpha=0.5, density=True, label='train')
    ax.hist(test[f], bins=bins, alpha=0.5, density=True, label='test')
    ax.set_xlabel(f)
    if ax == axs[0, 0]: ax.legend(loc='lower left')
        
    ax2 = ax.twinx()
    total, _ = np.histogram(train[f], bins=bins)
    failures, _ = np.histogram(train[f][train.failure == 1], bins=bins)
    with warnings.catch_warnings(): # ignore divide by zero for empty bins
        warnings.filterwarnings('ignore', category=RuntimeWarning)
        ax2.scatter((bins[1:] + bins[:-1]) / 2, failures / total,
                    color='m', s=10, label='failure probability')
    ax2.set_ylim(0, 0.5)
    ax2.tick_params(axis='y', colors='m')
    if ax == axs[0, 0]: ax2.legend(loc='upper right')
plt.tight_layout(w_pad=1)
plt.suptitle('Train and test distributions of the engineered features', fontsize=20, y=1.02)
plt.show()

In [ ]:
# Check how AUC changes when adding more variables sorted by fisher score importance
fs_scores = []
top_n_vars = 20
for i in range(1, top_n_vars+1):
    if i % 10 == 0: print('Added # top vars :', i)
    top_n_predictors = fs_df['predictor'][:i]
    clf = LogisticRegression(max_iter = 2500)
    fs_scores.append(cross_validate(clf, train[top_n_predictors], train['failure'],
                                    scoring='roc_auc', cv=5, verbose=0, n_jobs=-1, return_train_score=True))

# Plot
plt.plot([s['train_score'].mean() for s in fs_scores], color='blue')
plt.plot([s['test_score'].mean() for s in fs_scores], color='red')
plt.xlabel('# vars')
plt.ylabel('AUC')
plt.legend(['train', 'test'])
plt.show()

It is observed that the number of variables does not improve AUC score of the test set after 4 vars, in fact AUC decreases as we increase the number of variables. 

Classification + Regression

In [ ]:
seed = 42

random.seed(seed)
np.random.seed(seed)
tf.random.set_seed(seed)

In [ ]:
visible = Input(shape=(X_train.shape[1],))
layer = Dense(256, activation='swish', kernel_initializer='he_normal')(visible)
layer = Dense(128, activation='swish', kernel_initializer='he_normal')(layer)
layer = Dense(64, activation='swish', kernel_initializer='he_normal')(layer)
layer = Dense(32, activation='swish', kernel_initializer='he_normal')(layer)
layer = Dense(16, activation='swish', kernel_initializer='he_normal')(layer)
layer = Dense(8, activation='swish', kernel_initializer='he_normal')(layer)

# regression + classification
out_reg = Dense(1, activation='linear')(layer)
out_clas = Dense(2, activation='softmax')(layer)

model = Model(inputs=visible, outputs=[out_reg, out_clas])
model.compile(loss=['mse','sparse_categorical_crossentropy'], optimizer='adam')
plot_model(model, to_file='model.png', show_shapes=True)

In [ ]:
# %load_ext tensorboard

## Training
from datetime import datetime
from tensorflow import keras

callback = tf.keras.callbacks.EarlyStopping(monitor='loss', patience=5, restore_best_weights=True)

# Define the Keras TensorBoard callback.
logdir="../logs/fit/" + datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_callback = keras.callbacks.TensorBoard(log_dir=logdir)

# Train the model.
model.fit(X_train, [y_train,y_train_class], epochs=150, batch_size=32, callbacks=[callback], verbose=2)

### Data Cleaning

(https://www.kaggle.com/code/nnjjpp/adversarial-validation-detecting-data-drift)

In [ ]:
os.getcwd()

In [ ]:
import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt
import seaborn as sns

sample = pd.read_csv('../data/01_raw/tabular-playground-series-aug-2022/sample_submission.csv')
train = pd.read_csv('../data/01_raw/tabular-playground-series-aug-2022/train.csv')
test = pd.read_csv('../data/01_raw/tabular-playground-series-aug-2022/test.csv')

In [ ]:
# These modifications to the preprocessing steps allow for variable names to 
# be preserved, which is useful/needed when plotting variable importance

from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OrdinalEncoder
from sklearn.compose import ColumnTransformer
class SimpleImputerNamed(SimpleImputer):
    def get_feature_names_out(self):
        return list(self.feature_names_in_)
class OrdinalEncoderNamed(OrdinalEncoder):
    def get_feature_names_out(self):
        return list(self.feature_names_in_)
class ColumnTransformerNamed(ColumnTransformer):
    def get_feature_names_out(self):
        names = []
        for transformer in self.transformers_:
            if transformer[0] == 'remainder':
                if transformer[1] == 'passthrough':
                    names += list(self.feature_names_in_[transformer[2]])
                break
            else:
                names += transformer[1].get_feature_names_out()
        return names
    def fit(self, X, y=None):
        #print('In fit method')
        return super().fit(X,y)
    def transform(self, X):
        #print('In transform method')
        transformed = super().transform(X)
        return pd.DataFrame(transformed, columns= self.get_feature_names_out())
    def fit_transform(self, X, y=None):
        #print('In fit_transform method')
        fit_transformed = super().fit_transform(X,y)
        return pd.DataFrame(fit_transformed, columns=self.get_feature_names_out())

In [ ]:
#encoding categorical variable attribute_0 and attribute_1
#drop id - no prediction values
#when setting up hyperparameter tuning, possible to include product code as grouping variable.
#cross-method validation basis the disjoint product codes in train and test set should be GroupKFold.

In [ ]:
target = train.pop('failure')

In [ ]:
train.columns

In [ ]:
combined_data = pd.concat([train.drop(['id','failure'], axis=1),
                           test.drop(['id'], axis=1)])
combined_data_original = combined_data.copy()

missing_cols = combined_data.columns[list(combined_data.isna().sum()>0)]
missing_cols_original = missing_cols.copy()

categorical_cols = ['product_code', 'attribute_0', 'attribute_1']
categorical_cols_original = categorical_cols.copy()

preprocessing = ColumnTransformerNamed([('median_infill', SimpleImputerNamed(strategy='median'), missing_cols),
                                        ('ordinal_encode', OrdinalEncoderNamed(), categorical_cols)],
                                       remainder='passthrough')  

# Test the preprocessing pipeline/transformer:
preprocessing.fit_transform(combined_data)

In [ ]:
missing_cols = list(train.columns[train.isna().sum()>0])
categorical_cols = ['attribute_0', 'attribute_1']
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OrdinalEncoder

preprocessing = ColumnTransformer([('median_infill', SimpleImputer(strategy='median'), missing_cols),
                                   ('ordinal_encode', OrdinalEncoder(handle_unknown='use_encoded_value', unknown_value = -1), categorical_cols)],
                                  remainder='passthrough')

# Test the preprocessing pipeline/transformer:
preprocessing.fit_transform(train.drop(['product_code'], axis=1))
preprocessing.transform(test.drop(['product_code'], axis=1))

In [ ]:
# Test whether each row is a train or test. 
in_test_dataset = pd.DataFrame({'test': [0, ]*train.shape[0] + [1,]*test.shape[0]})

from xgboost import XGBRegressor
from sklearn.pipeline import Pipeline
from sklearn.metrics import roc_auc_score

mod_xgb = Pipeline(steps = (
    ['preprocessing', preprocessing],
    ['xgboost', XGBRegressor(objective = 'binary:logistic',
                             eval_metric = 'auc', 
                             random_state = 200)])
                   )

In [ ]:
def fit_adversarial_model(X, y, estimator_pipeline, plot=True):
    from sklearn.model_selection import KFold, cross_validate
    from sklearn.metrics import make_scorer, roc_auc_score
    cv_results = cross_validate(estimator=estimator_pipeline, 
                                X = X, y=y,
                                scoring = make_scorer(roc_auc_score),
                                cv = KFold(n_splits=5, random_state = 123, shuffle=True))
    mean_cv_score = cv_results['test_score'].mean()
    estimator_pipeline.fit(X, y)
    print(f'Area under ROC curve (cross-validated): {mean_cv_score:.2f}')

    print('\nVariable        Importance')
    print('--------------------------')
    try:
        column_heights = estimator_pipeline._final_estimator.feature_importances_
    except AttributeError: # use absolute value of coefficients if model doesn't support feature importances
        column_heights = np.abs(estimator_pipeline.steps[-1][1].coef_[0])
    for x in zip(estimator_pipeline.named_steps['preprocessing'].get_feature_names_out() ,
                 column_heights):
        print(f'{x[0]:<15} {x[1]:.2f}')
        
    if plot:
        
        x1,y1 = zip(*filter(lambda z:z[1] > 0, 
                     zip(estimator_pipeline.named_steps['preprocessing'].get_feature_names_out() ,
                         column_heights)))
        sns.barplot(x=list(x1), y=np.array(y1))
        plt.gca().tick_params(axis='x', rotation=90)

fit_adversarial_model(combined_data, in_test_dataset, estimator_pipeline = mod_xgb, plot=False)

In [ ]:
# Since there are no common values of product_code in the training and test data 
# (i.e. this variable is mutually exclusive), the 
# adversarial validation model discovered that we can just separate the different classes of 
# product_code to tell the training and test datasets apart. 
# Let's drop this variable and see what other variables the model can come up with

In [ ]:
try:
    categorical_cols.remove('product_code')
except ValueError:
    pass

combined_data_no_product_code = combined_data.drop(['product_code'], axis=1)

fit_adversarial_model(combined_data_no_product_code, in_test_dataset, estimator_pipeline = mod_xgb)

In [ ]:
# Since we've already saw this in EDA: `attribute_1`, `attribute_2` and `attribute_3`. Let's drop it
try:
    categorical_cols.remove('attribute_1')
except ValueError:
    pass
combined_data2 = combined_data_no_product_code.drop(['attribute_1', 'attribute_2', 'attribute_3'], axis=1)
fit_adversarial_model(combined_data2, in_test_dataset, estimator_pipeline = mod_xgb)

<iframe src="https://www.kaggle.com/embed/nnjjpp/adversarial-validation-detecting-data-drift?cellIds=23&kernelSessionId=103205636" height="300" style="margin: 0 auto; width: 100%; max-width: 950px;" frameborder="0" scrolling="auto" title="Adversarial validation - detecting data drift"></iframe>

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler

# Set up the pipeline again as we changed the input data (categorical_cols, etc.)
combined_data = combined_data_original.copy()
categorical_cols = categorical_cols_original.copy()
missing_cols = missing_cols_original.copy()
preprocessing = ColumnTransformerNamed([('median_infill', SimpleImputerNamed(strategy='median'), missing_cols),
                                        ('ordinal_encode', OrdinalEncoderNamed(), categorical_cols)],
                                       remainder='passthrough')  
mod_logreg = Pipeline(steps = (['preprocessing', preprocessing],
                               ['scaling', StandardScaler()],
                               ['logistic_regression', LogisticRegression(penalty='l2',
                                                                          solver='saga',
                                                                          class_weight='balanced',
                                                                          max_iter=2000,
                                                                          C=10**6,
                                                                          random_state = 200)]))

fit_adversarial_model(combined_data, np.array(in_test_dataset).ravel(), estimator_pipeline = mod_logreg, plot=True)

<iframe src="https://www.kaggle.com/embed/nnjjpp/adversarial-validation-detecting-data-drift?cellIds=26&kernelSessionId=103205636" height="300" style="margin: 0 auto; width: 100%; max-width: 900px;" frameborder="0" scrolling="auto" title="Adversarial validation - detecting data drift"></iframe>